In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
from sklearn.utils import shuffle
import nibabel as nib 
import glob
from matplotlib import pyplot as plt
%matplotlib inline
!pip install nilearn
import nilearn as nil
import cv2

     |████████████████████████████████| 2.5MB 2.8MB/s 


In [0]:
corr_imgs = sorted(glob.glob('drive/My Drive/Research/Augmented Dataset/Corrected Dataset/*'))   #reading corr mris
labels = sorted(glob.glob('drive/My Drive/Research/Augmented Dataset/Model_Output_Bias_images/*')) # reading labels

In [0]:
corr_imgs

In [0]:
labels = labels[20:]
len(labels)

In [0]:
labels

In [0]:
N = 64
total_imgs = 800

# Take each labels multiply with 40 skull stripped bias corrected images

for l in range(len(labels)): # Iterating through each label
  label = []
  a = nib.load(labels[l])
  a = a.get_fdata()
  for k in range(a.shape[0]):                                                                                                  
    temp = cv2.resize(a[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    label.append(temp)
  
  label = np.asarray(label)  
  label = label.reshape(-1,N,N,1)  # (260 * 64 * 64 * 1)

  for c in range(len(corr_imgs)): # Iterating through corrected images
    image = []
    a = nib.load(corr_imgs[c])
    a = a.get_fdata()
    for k in range(a.shape[0]):                                                                                                  
      temp = cv2.resize(a[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)                                          
      image.append(temp)
    image = np.asarray(image) 
    image = image.reshape(-1,N,N,1)  # (260 * 64 * 64 * 1)

    for i in range(len(label)):  
      for j in range(N):
        for k in range(N):
          if (image[i,j,k,0] == 0):  # Based on Skull Stripped image, making the bias zero
            label[i,j,k,0] = 0  

    temp_img = np.multiply(label,image) # Element wise multiplication of bias and corrected MRI  

    final_img = np.empty([260,311,260]) # converting the image from (260 * 64 * 64 * 1) to (260*311*260)
    for idx in range(0,260):
      img = temp_img[idx, :, :,0]
      img_resized = cv2.resize(img, (260, 311), interpolation=cv2.INTER_CUBIC)
      final_img[idx,:,:] = img_resized

    total_imgs = total_imgs + 1

    # Saving Images in respective folder
    final_img = nil.image.new_img_like(nib.load(corr_imgs[c]),final_img,copy_header=True)
    nib.save(final_img,'drive/My Drive/Research//Augmented Dataset/Aug_Inputs/' + 'input-' + str(total_imgs) + '.nii.gz')

    label_save = nib.load(labels[l]).get_fdata()
    label_save = nil.image.new_img_like(nib.load(labels[l]),label_save,copy_header=True)
    nib.save(label_save,'drive/My Drive/Research/Augmented Dataset/Aug_Labels/' + 'label-'  + str(total_imgs) +'.nii.gz')
  


In [0]:
label_save = nib.load(corr_imgs[0]).get_fdata()
label_save = nil.image.new_img_like(nib.load(labels[0]),label_save,copy_header=True)
nib.save(label_save,'drive/My Drive/Research/Augmented Dataset/original'  + str(total_imgs) +'.nii.gz')

In [0]:
list(range(len(labels)))

In [0]:
labels = labels[20:]

In [0]:
len(labels)

In [0]:
imgs = sorted(glob.glob('drive/My Drive/Research/Augmented Dataset/Aug_Inputs/*'))
labels = sorted(glob.glob('drive/My Drive/Research/Augmented Dataset/Aug_Labels/*'))

In [0]:
for l in range(len(labels)): # Iterating through each label
  label = []
  a = nib.load(labels[l])
  a = a.get_fdata()
  for k in range(a.shape[0]):                                                                                                  
    temp = cv2.resize(a[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    label.append(temp)

  label = np.asarray(label)
  label = label.reshape(-1,N,N,1) 

  np.savez_compressed('drive/My Drive/Research/Augmented Dataset/Aug_Labels_npz/label-'+ str(l+1) + '.npz', label)

In [0]:
np.shape(label)

In [0]:
t = np.load('drive/My Drive/Research/Augmented Dataset/Aug_Labels_npz/label-0.npz')['arr_0']

In [0]:
labels[1]

In [0]:
for l in range(len(imgs)): # Iterating through each label
  image = []
  a = nib.load(imgs[l])
  a = a.get_fdata()
  for k in range(a.shape[0]):                                                                                                  
    temp = cv2.resize(a[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)                                            
    image.append(temp)

  image = np.asarray(image)
  image = image.reshape(-1,N,N,1) 
  print(l)

  # np.savez_compressed('drive/My Drive/Research/Augmented Dataset/Aug_Inputs_npz/input-'+ str(l+1) + '.npz', image)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


KeyboardInterrupt: ignored

In [0]:
for f in range(1600):
      
      temp = nib.load(batch_y[f])['arr_0']